<a href="https://colab.research.google.com/github/zdanielshi/data_science_LFZ/blob/main/DS_0716_Home_Credit_Kaggle_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [140]:
train = pd.read_csv('/content/application_train.csv')
test = pd.read_csv('/content/application_test.csv')

# Analysis

In [141]:
train.keys()

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [142]:
corr_rank = train.corr()['TARGET'].sort_values()

In [143]:
corr_rank.head(n=15)

EXT_SOURCE_3                 -0.178919
EXT_SOURCE_2                 -0.160472
EXT_SOURCE_1                 -0.155317
DAYS_EMPLOYED                -0.044932
FLOORSMAX_AVG                -0.044003
FLOORSMAX_MEDI               -0.043768
FLOORSMAX_MODE               -0.043226
AMT_GOODS_PRICE              -0.039645
REGION_POPULATION_RELATIVE   -0.037227
ELEVATORS_AVG                -0.034199
ELEVATORS_MEDI               -0.033863
FLOORSMIN_AVG                -0.033614
FLOORSMIN_MEDI               -0.033394
LIVINGAREA_AVG               -0.032997
LIVINGAREA_MEDI              -0.032739
Name: TARGET, dtype: float64

In [144]:
corr_rank.tail(n=15)

DEF_60_CNT_SOCIAL_CIRCLE       0.031276
DEF_30_CNT_SOCIAL_CIRCLE       0.032248
LIVE_CITY_NOT_WORK_CITY        0.032518
OWN_CAR_AGE                    0.037612
DAYS_REGISTRATION              0.041975
FLAG_DOCUMENT_3                0.044346
REG_CITY_NOT_LIVE_CITY         0.044395
FLAG_EMP_PHONE                 0.045982
REG_CITY_NOT_WORK_CITY         0.050994
DAYS_ID_PUBLISH                0.051457
DAYS_LAST_PHONE_CHANGE         0.055218
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_BIRTH                     0.078239
TARGET                         1.000000
Name: TARGET, dtype: float64

# Feature Selection
* Make life simple on myself, just select the most features.

In [145]:
train = train[['TARGET', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'DEF_30_CNT_SOCIAL_CIRCLE']]

In [146]:
train

,TARGET,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_EMPLOYED,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_ID_PUBLISH,DEF_30_CNT_SOCIAL_CIRCLE
0,1,0.139376,0.083037,0.262949,-637,-9461,2,2,-1134.0,-2120,2.0
1,0,NaN,0.311267,0.622246,-1188,-16765,1,1,-828.0,-291,0.0
2,0,0.729567,NaN,0.555912,-225,-19046,2,2,-815.0,-2531,0.0
3,0,NaN,NaN,0.650442,-3039,-19005,2,2,-617.0,-2437,0.0
4,0,NaN,NaN,0.322738,-3038,-19932,2,2,-1106.0,-3458,0.0
...,...,...,...,...,...,...,...,...,...,...,...
307506,0,NaN,0.145570,0.681632,-236,-9327,1,1,-273.0,-1982,0.0
307507,0,NaN,NaN,0.115992,365243,-20775,2,2,0.0,-4090,0.0
307508,0,0.218859,0.744026,0.535722,-7921,-14966,3,3,-1909.0,-5150,0.0
307509,1,0.661024,NaN,0.514163,-4786,-11961,2,2,-322.0,-931,0.0


In [147]:
test = test[['EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH', 'DEF_30_CNT_SOCIAL_CIRCLE']]

In [148]:
test

,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_EMPLOYED,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_ID_PUBLISH,DEF_30_CNT_SOCIAL_CIRCLE
0,0.159520,0.752614,0.789654,-2329,-19241,2,2,-1740.0,-812,0.0
1,0.432962,0.564990,0.291656,-4469,-18064,2,2,0.0,-1623,0.0
2,0.610991,NaN,0.699787,-4458,-20038,2,2,-856.0,-3503,0.0
3,0.612704,0.525734,0.509677,-1866,-13976,2,2,-1805.0,-4208,0.0
4,NaN,0.202145,0.425687,-2191,-13040,2,2,-821.0,-4262,0.0
...,...,...,...,...,...,...,...,...,...,...
48739,0.643026,NaN,0.648575,-5169,-19970,3,3,-684.0,-3399,0.0
48740,NaN,NaN,0.684596,-1149,-11186,2,2,0.0,-3003,0.0
48741,0.283712,0.733503,0.632770,-3037,-15922,2,2,-838.0,-1504,0.0
48742,0.595456,0.373090,0.445701,-2731,-13968,2,2,-2308.0,-1364,0.0


# Stacking the notebooks

In [149]:
train['TRAINING'] = True
test['TRAINING'] = False

In [150]:
y = train.TARGET

In [151]:
y

0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64

In [152]:
train.drop(columns = 'TARGET', inplace=True)

In [153]:
x = pd.concat([train, test], axis = 0)

# Transformations

In [154]:
x.skew()

EXT_SOURCE_3                  -0.398628
EXT_SOURCE_1                  -0.077242
EXT_SOURCE_2                  -0.793076
DAYS_EMPLOYED                  1.652128
DAYS_BIRTH                    -0.115918
REGION_RATING_CLIENT_W_CITY    0.052475
REGION_RATING_CLIENT           0.081122
DAYS_LAST_PHONE_CHANGE        -0.711807
DAYS_ID_PUBLISH                0.338329
DEF_30_CNT_SOCIAL_CIRCLE       8.111785
TRAINING                      -2.113585
dtype: float64

# Scaling

In [155]:
cols_to_scale = x.select_dtypes(exclude = ['object', 'bool']).columns.values.tolist()
cols_to_scale

['EXT_SOURCE_3',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'DAYS_EMPLOYED',
 'DAYS_BIRTH',
 'REGION_RATING_CLIENT_W_CITY',
 'REGION_RATING_CLIENT',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_ID_PUBLISH',
 'DEF_30_CNT_SOCIAL_CIRCLE']

In [156]:
robust_scaler = RobustScaler().fit(x[cols_to_scale])
standard_scaler = StandardScaler().fit(x[cols_to_scale])
minmax_scaler = MinMaxScaler().fit(x[cols_to_scale])

In [157]:
x[cols_to_scale] = robust_scaler.transform(x[cols_to_scale])

In [158]:
x

,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_EMPLOYED,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_ID_PUBLISH,DEF_30_CNT_SOCIAL_CIRCLE,TRAINING
0,-1.327464,-1.252419,-1.126607,0.235648,0.868018,0.0,0.0,-0.277948,0.435217,2.0,True
1,NaN,-0.576862,0.214188,0.014452,-0.139291,-1.0,-1.0,-0.043645,1.138408,0.0,True
2,0.660472,NaN,-0.033351,0.401044,-0.453868,0.0,0.0,-0.033691,0.277201,0.0,True
3,NaN,NaN,0.319407,-0.728623,-0.448214,0.0,0.0,0.117917,0.313341,0.0,True
4,NaN,NaN,-0.903489,-0.728222,-0.576058,0.0,0.0,-0.256508,-0.079200,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...
48739,0.368976,NaN,0.312441,-1.583701,-0.581299,1.0,1.0,0.066616,-0.056517,0.0,False
48740,NaN,NaN,0.446860,0.030108,0.630120,0.0,0.0,0.590352,0.095732,0.0,False
48741,-0.841298,0.672947,0.253460,-0.727820,-0.023031,0.0,0.0,-0.051302,0.672049,0.0,False
48742,0.208749,-0.393868,-0.444627,-0.604978,0.246449,0.0,0.0,-1.176876,0.725875,0.0,False


# Encoding

In [159]:
x = pd.get_dummies(x)

# Imputing Nulls

In [160]:
# impute_n = 3
# imputer = KNNImputer(n_neighbors = impute_n)

In [161]:
imputer = IterativeImputer()

In [162]:
x.isna().sum()

EXT_SOURCE_3                    69633
EXT_SOURCE_1                   193910
EXT_SOURCE_2                      668
DAYS_EMPLOYED                       0
DAYS_BIRTH                          0
REGION_RATING_CLIENT_W_CITY         0
REGION_RATING_CLIENT                0
DAYS_LAST_PHONE_CHANGE              1
DAYS_ID_PUBLISH                     0
DEF_30_CNT_SOCIAL_CIRCLE         1050
TRAINING                            0
dtype: int64

In [163]:
x = pd.DataFrame(imputer.fit_transform(x), columns = x.columns)

In [164]:
x

,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_EMPLOYED,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_ID_PUBLISH,DEF_30_CNT_SOCIAL_CIRCLE,TRAINING
0,-1.327464,-1.252419,-1.126607,0.235648,0.868018,0.0,0.0,-0.277948,0.435217,2.0,1.0
1,-0.381782,-0.576862,0.214188,0.014452,-0.139291,-1.0,-1.0,-0.043645,1.138408,0.0,1.0
2,0.660472,0.475703,-0.033351,0.401044,-0.453868,0.0,0.0,-0.033691,0.277201,0.0,1.0
3,0.021491,0.438632,0.319407,-0.728623,-0.448214,0.0,0.0,0.117917,0.313341,0.0,1.0
4,0.003271,0.414266,-0.903489,-0.728222,-0.576058,0.0,0.0,-0.256508,-0.079200,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
356250,0.368976,0.438520,0.312441,-1.583701,-0.581299,1.0,1.0,0.066616,-0.056517,0.0,0.0
356251,-0.229323,-0.324650,0.446860,0.030108,0.630120,0.0,0.0,0.590352,0.095732,0.0,0.0
356252,-0.841298,0.672947,0.253460,-0.727820,-0.023031,0.0,0.0,-0.051302,0.672049,0.0,0.0
356253,0.208749,-0.393868,-0.444627,-0.604978,0.246449,0.0,0.0,-1.176876,0.725875,0.0,0.0


In [166]:
x.isna().sum().sum()

0

# Unstacking the train and test sets

In [167]:
train = x[x['TRAINING'] == True]

In [168]:
test = x[x['TRAINING'] == False]

In [171]:
train.drop(columns = 'TRAINING', inplace=True)
test.drop(columns = 'TRAINING', inplace=True)

KeyError: ignored

# SMOTE

In [172]:
smote = SMOTE()

In [173]:
x_smote, y_smote = smote.fit_resample(train, y)

# Training Data

In [174]:
xtrain, xtest, ytrain, ytest = train_test_split(x_smote, y_smote, test_size = .3, random_state = 7)

# KNN Classifier

In [175]:
n = 7
knn = KNeighborsClassifier(n_neighbors = n).fit(xtrain, ytrain)

In [176]:
knn_predict = knn.predict(xtest)

In [177]:
knn_score = knn.score(xtest, ytest)
knn_score

0.8347109874301346

In [178]:
knn_recall = recall_score(ytest, knn_predict)
knn_precision = precision_score(ytest, knn_predict)

In [179]:
knn_score, knn_recall, knn_precision

(0.8347109874301346, 0.9873756109050227, 0.7566648015450427)

In [215]:
knn_rocauc = roc_auc_score(ytest, knn.predict_proba(xtest)[:,1])

In [216]:
knn_rocauc

0.9418990271704688

# Logistic Regression

In [180]:
log = LogisticRegression().fit(xtrain, ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [181]:
log_predict = log.predict(xtest)

In [182]:
log_score = log.score(xtest, ytest)
log_recall = recall_score(ytest, log_predict)
log_precision = precision_score(ytest, log_predict)

In [183]:
log_score, log_recall, log_precision

(0.6680364596844562, 0.6581522699169758, 0.6720095233514501)

In [213]:
log_rocauc = roc_auc_score(ytest, log.predict_proba(xtest)[:,1])

In [214]:
log_rocauc

0.7294299082222677

# LGBM Classifier

In [184]:
lgbm = LGBMClassifier()
lgbm.fit(xtrain, ytrain)

LGBMClassifier()

In [185]:
lgbm_predict = lgbm.predict(xtest)

In [186]:
lgbm_score = lgbm.score(xtest, ytest)
lgbm_recall = recall_score(ytest, lgbm_predict)
lgbm_precision = precision_score(ytest, lgbm_predict)

In [187]:
lgbm_score, lgbm_recall, lgbm_precision

(0.7064948234794708, 0.706612494847789, 0.7069788266899176)

In [218]:
lgbm_rocauc = roc_auc_score(ytest, lgbm.predict_proba(xtest)[:,1])

In [219]:
lgbm_rocauc

0.7822789449147491

# Summarized results

In [220]:
print('{:.4f}'.format(knn_score), '- KNN Accuracy w/', n, 'neighbors')
print('{:.4f}'.format(knn_recall), '- KNN Recall w/', n, 'neighbors')
print('{:.4f}'.format(knn_precision), '- KNN Precision w/', n, 'neighbors')
print('{:.4f}'.format(knn_rocauc), '- KNN ROC AUC w/', n, 'neighbors')
print()
print('{:.4f}'.format(log_score), '- Log Accuracy')
print('{:.4f}'.format(log_recall), '- Log Recall')
print('{:.4f}'.format(log_precision), '- Log Precision')
print('{:.4f}'.format(log_rocauc), '- Log ROC AUC')
print()
print('{:.4f}'.format(lgbm_score), '- LGBM Accuracy')
print('{:.4f}'.format(lgbm_recall), '- LGBM Recall')
print('{:.4f}'.format(lgbm_precision), '- LGBM Precision')
print('{:.4f}'.format(lgbm_rocauc), '- LGBM ROC AUC')

0.8347 - KNN Accuracy w/ 7 neighbors
0.9874 - KNN Recall w/ 7 neighbors
0.7567 - KNN Precision w/ 7 neighbors
0.9419 - KNN ROC AUC w/ 7 neighbors

0.6680 - Log Accuracy
0.6582 - Log Recall
0.6720 - Log Precision
0.7294 - Log ROC AUC

0.7065 - LGBM Accuracy
0.7066 - LGBM Recall
0.7070 - LGBM Precision
0.7823 - LGBM ROC AUC


# Predicting

In [193]:
submit = knn.predict(test)

In [201]:
submission = pd.read_csv('/content/application_test.csv')

In [203]:
submission = submission[['SK_ID_CURR']]

In [205]:
submission['TARGET'] = submit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [206]:
submission

,SK_ID_CURR,TARGET
0,100001,0
1,100005,0
2,100013,0
3,100028,1
4,100038,1
...,...,...
48739,456221,0
48740,456222,0
48741,456223,0
48742,456224,0


In [209]:
submission.to_csv('submission (KNN)', index=False)